# Full Interactive Dashboard — MainPlotGrid with Coordinated Controls

This notebook combines multiple widget types into a single dashboard that drives a `MainPlotGrid` (the three-panel composite that mirrors the iSLAT GUI layout):

- **Molecule dropdown** — selects the active molecule for the inspection and population panels
- **Range slider** — controls the line inspection wavelength window
- **Parameter sliders** — adjust T, N, and R for the selected molecule in real time

All controls are coordinated: changing the active molecule updates the parameter sliders to reflect that molecule's current values.

In [ ]:
# First, add the iSLAT package to the Python path
import sys
from pathlib import Path

# Navigate from notebook location to the iSLATTests directory (where iSLAT package lives)
notebook_dir = Path.cwd()
islat_root = notebook_dir.parent.parent.parent  # Interactive Widgets -> Notebooks -> Examples -> iSLAT
if str(islat_root) not in sys.path:
    sys.path.insert(0, str(islat_root))

# Core libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Use the interactive widget backend for matplotlib
%matplotlib widget

# IPython widget libraries
import ipywidgets as widgets
from IPython.display import display, clear_output

# iSLAT data types
from iSLAT.Modules.DataTypes import Molecule, MoleculeDict

# iSLAT standalone plot classes
from iSLAT.Modules.Plotting import (
    BasePlot,
    DEFAULT_THEME,
    LineInspectionPlot,
    PopulationDiagramPlot,
    FullSpectrumPlot,
    MainPlotGrid,
)

print("Imports successful!")
print(f"matplotlib backend: {matplotlib.get_backend()}")

In [ ]:
# --- Load observed data and define a molecule factory ---
import pandas as pd

# Load CI Tau MIRI spectrum (Banzatti+2023b)
data_path = Path(islat_root) / "iSLAT" / "DATAFILES" / "EXAMPLE-data" / "CITau_MIRI_Banzatti+2023b.csv"
obs = pd.read_csv(data_path)

wave_grid     = obs["wave"].values
observed_flux = obs["flux"].values
observed_err  = obs["err"].values
continuum     = obs["cont"].values

print(f"Loaded: {data_path.name}")
print(f"  {len(wave_grid)} points, {wave_grid.min():.2f}–{wave_grid.max():.2f} μm")

# Paths to HITRAN parameter files
data_dir = Path(islat_root) / "iSLAT" / "DATAFILES" / "HITRANdata"
water_par_file = str(data_dir / "data_Hitran_H2O.par")
co_par_file    = str(data_dir / "data_Hitran_CO.par")
co2_par_file   = str(data_dir / "data_Hitran_CO2.par")

# Wavelength range derived from observed data
wavelength_range = (float(wave_grid.min()), float(wave_grid.max()))

# Default molecule definitions (shared across all sections)
DEFAULT_MOLECULES = {
    "H2O": {"Molecule Name": "H2O", "temp": 850, "n_mol": 1e18, "radius": 0.5, "color": "#0000FF",
             "displaylabel": "$H_2O$", "File Path": water_par_file},
    "CO":  {"Molecule Name": "CO",  "temp": 1000, "n_mol": 1e18, "radius": 0.4, "color": "#FF0000",
             "displaylabel": "CO",  "File Path": co_par_file},
    "CO2": {"Molecule Name": "CO2", "temp": 300,  "n_mol": 1e17, "radius": 0.5, "color": "green",
             "displaylabel": "$CO_2$", "File Path": co2_par_file},
}

def create_mol_dict():
    """
    Create a fresh, independent MoleculeDict with the default molecules.
    """
    md = MoleculeDict(
        global_distance=160,
        global_stellar_rv=0.0,
        global_wavelength_range=wavelength_range,
        global_model_pixel_res=0.0013,
    )
    md.load_molecules(
        molecules_data=[v for v in DEFAULT_MOLECULES.values()],
        initial_molecule_parameters=DEFAULT_MOLECULES,
    )
    md.bulk_update_parameters({"fwhm": 130, "broad": 1})
    return md

# Quick sanity check
_test = create_mol_dict()
print(f"create_mol_dict() → {list(_test.keys())}")
del _test

In [ ]:
# --- Full interactive dashboard with MainPlotGrid ---

import math

mol_dict_5 = create_mol_dict()

# Create the three-panel grid
grid = MainPlotGrid(
    wave_data=wave_grid,
    flux_data=observed_flux,
    molecules=mol_dict_5,
    active_molecule=mol_dict_5["H2O"],
    inspection_range=(14.0, 17.0),
    figsize=(16, 9),
)
grid.generate_plot()

# ── Widgets ──────────────────────────────────────────────────────────────

# Molecule selector
dash_mol_dropdown = widgets.Dropdown(
    options=list(mol_dict_5.keys()),
    value="H2O",
    description='Active molecule:',
    style={'description_width': '120px'},
)

# Inspection wavelength range
dash_range_slider = widgets.FloatRangeSlider(
    value=[14.0, 17.0],
    min=float(wave_grid.min()),
    max=float(wave_grid.max()),
    step=0.1,
    description='Inspect λ (μm):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='600px'),
)

# Temperature slider
dash_temp_slider = widgets.IntSlider(
    value=int(mol_dict_5["H2O"].temp), min=100, max=2000, step=50,
    description='T (K):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='450px'),
)

# log₁₀ column density
dash_logn_slider = widgets.FloatSlider(
    value=math.log10(mol_dict_5["H2O"].n_mol), min=14, max=22, step=0.1,
    description='log₁₀ N (cm⁻²):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='450px'),
    readout_format='.1f',
)

# Emitting radius
dash_radius_slider = widgets.FloatSlider(
    value=mol_dict_5["H2O"].radius, min=0.05, max=5.0, step=0.05,
    description='Radius (AU):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='450px'),
)

# Status label
dash_status = widgets.HTML(value="<i>Ready</i>")

# ── Callbacks ────────────────────────────────────────────────────────────

_updating_sliders = False  # guard to prevent feedback loops

def sync_sliders_to_molecule(mol):
    """Set slider values to reflect the current molecule's parameters."""
    global _updating_sliders
    _updating_sliders = True
    dash_temp_slider.value = int(mol.temp)
    dash_logn_slider.value = round(math.log10(mol.n_mol), 1) if mol.n_mol > 0 else 14.0
    dash_radius_slider.value = mol.radius
    _updating_sliders = False

def on_dash_mol_change(change):
    """Switch active molecule and update slider positions."""
    mol_name = change['new']
    mol = mol_dict_5[mol_name]
    sync_sliders_to_molecule(mol)
    grid.set_active_molecule(mol)
    grid.fig.canvas.draw_idle()
    dash_status.value = f"<i>Active: {mol_name}</i>"

def on_dash_range_change(change):
    """Update inspection range in the grid."""
    xmin, xmax = change['new']
    grid.set_inspection_range(xmin, xmax)
    grid.fig.canvas.draw_idle()

def on_dash_param_change(change):
    """Update the active molecule's parameters and refresh all panels."""
    if _updating_sliders:
        return
    mol_name = dash_mol_dropdown.value
    mol = mol_dict_5[mol_name]
    mol.temp = float(dash_temp_slider.value)
    mol.n_mol = 10 ** dash_logn_slider.value
    mol.radius = dash_radius_slider.value
    grid.refresh()
    grid.fig.canvas.draw_idle()
    dash_status.value = (
        f"<i>{mol_name}: T={mol.temp:.0f} K, "
        f"N={mol.n_mol:.1e} cm⁻², R={mol.radius:.2f} AU</i>"
    )

# Wire up observers
dash_mol_dropdown.observe(on_dash_mol_change, names='value')
dash_range_slider.observe(on_dash_range_change, names='value')
dash_temp_slider.observe(on_dash_param_change, names='value')
dash_logn_slider.observe(on_dash_param_change, names='value')
dash_radius_slider.observe(on_dash_param_change, names='value')

# ── Layout ───────────────────────────────────────────────────────────────
left_col = widgets.VBox([dash_mol_dropdown, dash_range_slider])
right_col = widgets.VBox([dash_temp_slider, dash_logn_slider, dash_radius_slider])
controls = widgets.VBox([
    widgets.HBox([left_col, right_col]),
    dash_status,
])

display(controls)
grid.show()